# Práctica 1 - Self-Organising Maps - COLORES
## Preparación de entorno
#### Importar librerías de código

In [262]:
# from __future__ import division

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as patches
import pandas as pd
from sklearn.preprocessing import StandardScaler

%matplotlib inline

#### Dataset que se va a utilizar para el entrenamiento

In [263]:
# Código para obtener el Dataset que se va a usar en el entrenamiento
datosOriginal = pd.read_csv("pokemon_train.csv", encoding = 'latin_1', sep = ',')
datosDF = datosOriginal
nombres_pokemon = datosDF.pop("name")
tipos_pokemon_type1 = datosDF.pop("type1")
tipos_pokemon_type2 = datosDF.pop("type2")
id_pokemon = datosDF.pop("id")

datosClasificarDF = pd.read_csv("pokemon_classify.csv", encoding = 'latin_1', sep = ',')
datosClasificarDF = datosClasificarDF.drop(["name","type1","type2","id"], axis=1)

## SOM Setup
#### Variables definidas por el alumno

In [264]:
# Valores de entradas 
n_entradas_kohonen = len(datosDF.columns) # 22 entradas 
tam_dataset = len(datosDF) # 247 valores

In [265]:
# Inicializa tamaño del mapa de Kohonen, número de iteraciones y learning rate
# Inicializa normalizar_datos dependiendo de si tienes que normalizar los datos o no
lado_mapa = 30 # Cambiar
periodo = 500 # Cambiar
learning_rate = 0.15 # Cambiar
normalizar_datos = False

#### A partir de este punto solo hay cálculos. No se introducen más valores "a mano"

In [266]:
# Establece el numero de entradas del mapa y el número de datos que se van a usar para entrenar. 
# Utiliza una función que obtenga automáticamente los valores a partir del Dataset.
num_entradas = n_entradas_kohonen
num_datos = tam_dataset

# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario_inicial = int(lado_mapa/2)

# Normaliza los datos si fuese necesario dividiendo cada dato por el máximo en la matriz
if normalizar_datos:
    datosAR = datosDF.to_numpy()
    datosAR = (datosAR - datosAR.min()) / ( datosAR.max() - datosAR.min())
    
# Crea una matriz de pesos con valores random entre 0 y 1. Usa la función random.random de la librería NumPy
matriz_pesos = np.random.random((lado_mapa,lado_mapa,num_entradas))
matriz_pesos_original = matriz_pesos # La guardamos para poder comparar en un futuro

#### Funciones para entrenar/clasificar

In [267]:
# Funcion calcular BMU
def calcular_bmu(patron_entrada, m_pesos):
   # Obtenemos dimensiones de la matriz de pesos para los bucles
   rows = len(m_pesos)
   columns = len(m_pesos[0])
   # Decimos que la ganadora al principio sea la primera neurona
   ganadora = np.linalg.norm(np.array(m_pesos[0][0]) - np.array(patron_entrada))
   Bmu = m_pesos[0][0]
   BmuIndex = np.stack((0,0))
   # Bucle que recorre toda las neuronas
   for x in range(rows):
      for y in range(columns):
         # Obten la distancia euclidea entre la neurona y el patron
         dist = np.linalg.norm(np.array(m_pesos[x][y]) - np.array(patron_entrada))
         # Si la nueva distancia es menor que la ganadora asigna esta neurona como la nueva ganadora
         if dist < ganadora:
            ganadora = dist
            Bmu = m_pesos[x][y]
            BmuIndex = np.stack((x,y))
   return Bmu , BmuIndex



In [268]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
def variacion_learning_rate(lr_inicial, i, n_iteraciones):
   learning_rate = lr_inicial * (1 - i/n_iteraciones)
   return learning_rate


In [269]:
# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
   vecindario = 1 + vecindario_inicial * (1 - i/n_iteraciones)
   return vecindario
   

In [270]:
# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU**2 / (2*vecindario_actual**2))

In [271]:
#Funcion Ajuste de matriz de pesos
def ajustarPesos(IndexN , pokemon , learning_rate , matriz_pesos , vecindario):
    # Formula para obtener los pesos nuevos de la BMUW
    matriz_pesos[IndexN[0]][IndexN[1]] += (learning_rate * (pokemon - matriz_pesos[IndexN[0]][IndexN[1]]))
    # Obtenemos las dimensiones para el bucle
    rows = len(matriz_pesos)
    cols = len(matriz_pesos[0])
    # Bucle que recorre todas la neuronas incluida la BMU
    for x in range(rows):
        for y in range(cols):
            vectorNeurona = [x,y] #Guardamos el indice como un vector
            vectorBMU = [IndexN[0],IndexN[1]] #Guardamos el indice como un vecto
            #Calculamos la discacia euclidea entre ambos vectores para determinar como de lejos estan los indices
            dist_euc = np.linalg.norm(np.array(vectorBMU) - np.array(vectorNeurona)) 
            # Si la distancia es menor o igual que el vecindario y no es 0 (caso de la BMU) entonces sabemos que es vecina de BMU
            if(dist_euc <= vecindario and dist_euc != 0):
              # Dime segun la distancia entre la neuroma vecina y la bmu cuanto es el decay del learning rate
              dist_decay = decay(dist_euc,vecindario)
              # Aplico formula de ajuste de pesos a la neurona vecina segun los datos obtenidos
              matriz_pesos[x][y] += (learning_rate * dist_decay * (pokemon -  matriz_pesos[x][y]))  
    return matriz_pesos

## PINTAR HISTOGRAMA

## SOM Entrenamiento

In [272]:
# Entrena la red con el dataset de entrenamiento
# Guardamos las variables iniciales para no modificarlas en las iteraciones de entrenamiento
vecindario = vecindario_inicial
learning_rate_ajustado = learning_rate
for x in range(periodo):
    # Imprime la iteracion actual cada 100 iteraciones
    if(x % 100 == 0 and x != 0):
        print('Iteracion Nº: ',x,end="\r")
    #Obtenemos un indice aleatorio de nuestro set de pokemon
    numAleatorio = np.random.randint(0, tam_dataset)
    pokemon = datosAR[numAleatorio] 
    #Pasamos el pokemon aleatorio a calcular bmu para que nos devuelva la neurona mas proxima al patron
    Bmu , BmuIndex = calcular_bmu(pokemon,matriz_pesos)
    #Ajustamos la matrix de pesos segun los datos obtenidos
    matriz_pesos = ajustarPesos(BmuIndex , pokemon , learning_rate_ajustado , matriz_pesos , vecindario)
    #Una vez ajustada la matriz de pesos modificamos el learning rate para la siguiente iteracion
    learning_rate_ajustado = variacion_learning_rate(learning_rate, x , periodo)
    #Una vez ajustada la matriz de pesos modificamos el vecindario para la siguiente iteracion
    vecindario = variacion_vecindario(vecindario_inicial , x , periodo)

print('Iteracion Nº: ',x + 1)

Iteracion Nº:  500


## SOM Clasificación

In [273]:
# Clasifica los patrones de entrenamiento con la matriz de pesos recién entrenada

def Clasificacion(pokemon , matriz_pesos):

    Mapa_Clasificacion = np.zeros((lado_mapa,lado_mapa,num_entradas))
    Mapa_Activacion = np.zeros((lado_mapa,lado_mapa),dtype=int)
    Mapa_Distancias = np.zeros((lado_mapa,lado_mapa))

    clase = 0

    for poke in pokemon:
        bmu , bmuIndex = calcular_bmu(poke,matriz_pesos)
        Mapa_Activacion[bmuIndex[0],bmuIndex[1]] += 1 # Cada vez que se activa una neurona (x,y de la ganadora ++)
        #print("Neurona gandora: ", bmuIndex)
        Mapa_Clasificacion[bmuIndex[0],bmuIndex[1]] = poke # Se guarda el último color que ha activado la neurona
        # La media de las distancias euclidieas de esa neurona
        Mapa_Distancias[bmuIndex[0],bmuIndex[1]] = (Mapa_Distancias[bmuIndex[0],bmuIndex[1]] + np.linalg.norm(np.array(bmu) - np.array(poke))) / 2 
        
    for x in range(len(Mapa_Activacion)):
        for y in range(len(Mapa_Activacion[0])):
            if Mapa_Activacion[x][y] > 0:
                clase += 1

    print('Numero de Clases: ', clase)
    return Mapa_Activacion, Mapa_Clasificacion, Mapa_Distancias


Mapa_Activacion, Mapa_Clasificacion, Mapa_Distancias = Clasificacion(datosAR, matriz_pesos)


Numero de Clases:  54


## SOM Prueba

In [274]:
# Clasifica nuevos patrones

#datosClasificar = np.random.rand(18,100)
datosClasificar = datosAR
#datosClasificar = np.transpose(datosClasificar)

Mapa_Activacion, Mapa_Clasificacion, Mapa_Distancias = Clasificacion(datosClasificar, matriz_pesos)
# Funcion para calcular el error de cuantificacion dado un mapa de Distancias
def Calcular_err_cuantificacion(Mapa_Distancias):
    ListaDistancia = []
    for x in range(len(Mapa_Distancias)):
        for y in range(len(Mapa_Distancias[0])):
            if Mapa_Distancias[x][y] != 0:
             ListaDistancia.append(Mapa_Distancias[x][y])
    Media = sum(ListaDistancia) / len(ListaDistancia)
    return Media

# Funcion que devuelve si la segunda bmu es adyacente o no dado un patron
def calcular_err_topologico(patron_entrada, m_pesos):
    rows = len(m_pesos)
    columns = len(m_pesos[0])
    # Asignamos que la ganadora 1 sea la primera neurona y la ganadora 2 la segunda por tener un valor con cual empezar
    ganadora = np.linalg.norm(np.array(m_pesos[0][0]) - np.array(patron_entrada))
    ganadora2 = np.linalg.norm(np.array(m_pesos[0][1]) - np.array(patron_entrada))
    BmuIndex = np.stack((0,0))
    Bmu2Index = np.stack((0,1))
    # Mismo algoritmo que CalcularBMU pero en este se guarda la segunda BMU
    for x in range(rows):
        for y in range(columns):
            dist = np.linalg.norm(np.array(m_pesos[x][y]) - np.array(patron_entrada))
            if dist < ganadora:
                ganadora = dist
                BmuIndex = np.stack((x,y))
            else:
                if dist < ganadora2:
                    ganadora2 = dist
                    Bmu2Index = np.stack((x,y))
    # Devuelve 0 si es adyacente y 1 en el caso contrario
    return (0 if np.linalg.norm(np.array(BmuIndex)-np.array(Bmu2Index)) < 2 else 1)

# Aqui calculamos e imprimimos los errores
print('Error De Cuantificacion: ', Calcular_err_cuantificacion(Mapa_Distancias))
listaX = []

# Dado un set de colores guarda en una lista los valores devueltos
for pokemon in datosClasificar:
    listaX.append(calcular_err_topologico(pokemon,matriz_pesos))
# Calculamos la Media
media = sum(listaX) / len(listaX)
print('Error Topologico: ', media)

ValueError: operands could not be broadcast together with shapes (18,) (247,) 